In [2]:
from rosetta import *
init(notebook=True)
import random
from math import exp
from toolbox import pose_from_rcsb

core.init: Rosetta version  from 
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/XT/ResearchApps/PyRosetta_1230/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-1454737401 seed_offset=0 real_seed=-1454737401
core.init.random: RandomGenerator:init: Normal mode, seed=-1454737401 RG_type=mt19937


In [4]:
pose=Pose()
save_pose=Pose()

In [15]:
initial_pose=pose_from_sequence('A'*10)
pose.assign(initial_pose)

In [20]:
initial_pose=pose_from_sequence('G'*10)
pose.assign(initial_pose)

In [2]:
initial_pose=pose_from_rcsb('1B72')

The file 1B72.pdb already exists; this file will be overwritten.
PDB 1B72 successfully loaded from the RCSB into 1B72.pdb.
If the file 1B72.clean.pdb already exists,it will be overwritten.
PDB 1B72.pdb successfully cleaned, non-ATOM lines removed.
Clean data written to 1B72.clean.pdb.
core.chemical.ResidueTypeSet: Finished initializing fa_standard residue type set.  Created 384 residue types
core.chemical.ResidueTypeSet: Total time to initialize 0.509799 seconds.
core.io.pose_from_sfr.chirality_resolution: Flipping atom xyz for  H5' and H5'' for residue  DC
core.io.pose_from_sfr.chirality_resolution: Flipping atom xyz for  H21 and  H22 for residue  DG
core.io.pose_from_sfr.chirality_resolution: Number of flip-atom fixups exceeds output limit. Rerun with -show_all_fixes to show everything.
core.conformation.Conformation: [ WARNING ] Building missing atom ( P  ) at root of residue tree, using stubs:  O5'  C5'  C4'
This probably means that a torsion angle is being taken from the ideal res

In [27]:
initial_pose=pose_from_file('/Users/XT/Dropbox/1B72.aa_only.pdb')

core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ARG 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue ARG 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE  on residue ARG 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CZ  on residue ARG 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NH1 on residue ARG 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NH2 on residue ARG 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 8
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 8
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 8
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on residue LYS 8
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ASN 29
core.conformation.Conformation: [ WARNING ] missing heavyatom:  

In [28]:
pose.assign(initial_pose)

In [6]:
tot_resi=pose.total_residue()
print tot_resi

10


In [37]:
pmm=PyMolMover()

In [46]:
pmm.apply(pose)

In [7]:
scorefxn=ScoreFunction()   # I copied the weights from full-atom scorefxn
scorefxn.set_weight(fa_atr,0.8)
scorefxn.set_weight(fa_rep,0.44)
scorefxn.set_weight(hbond_sr_bb,1.17)
scorefxn.set_weight(hbond_lr_bb,1.17)
scorefxn.set_weight(hbond_bb_sc,1.17)
scorefxn.set_weight(hbond_sc,1.1)

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBEval.csv


In [ ]:
# 100, 100, 95 -444.281901131
# 100, 200, 34 -450.026655831
# 100, 300, 44 -435.795196009  # 200 enough?
num_of_decoy=100
iteration_num=500
f=open('10ALA.txt','w+')
low_score=0
for decoy_num in range(1,num_of_decoy+1):
    pose.assign(initial_pose)
    tot_resi=pose.total_residue()
    for i in range(iteration_num):
        save_pose.assign(pose)
        score=scorefxn(pose)
        old_score=score
        resi_num=random.randint(1,tot_resi)  # inclusive
        torsion=random.choice(['phi','psi'])
        if torsion=='phi':
            current_angle=pose.phi(resi_num)
            new_angle=random.gauss(current_angle,25)
            pose.set_phi(resi_num,new_angle)
        else:
            current_angle=pose.psi(resi_num)
            new_angle=random.gauss(current_angle,25)
            pose.set_psi(resi_num,new_angle)
        #pmm.apply(pose)

        # Monte Carlo
        score=scorefxn(pose)
        delta_E=score-old_score
        #print i+1, '\t', old_score, score, delta_E
        if delta_E>=0:
            P=exp(-delta_E)*100
            random_num=random.randint(1,100)
            if P<random_num:
                pose.assign(save_pose)
                score=scorefxn(pose)
                print 'reject', old_score,score,delta_E,P, random_num
            else:
                print ">0acc",old_score,score,delta_E,P, random_num
        else:
            print "accept",old_score,score,delta_E,P
                
    print decoy_num, score
    if score<low_score:
        low_score=score
        print 'new_low_score',decoy_num, low_score

    # phi psi
    for i in range(1,tot_resi+1):
        phi=pose.phi(i)
        psi=pose.psi(i)
        if phi>180:
            phi=phi-360
        if psi>180:
            psi=psi-360
        #print i,phi, psi
        f.write(str(decoy_num)+'\t'+str(phi)+'\t'+str(psi)+'\n')
f.close()





    

reject -375.891395077 -375.891395077 14.8338007518 3.61212015019e-05 17
reject -375.891395077 -375.891395077 209.637753311 9.02569260443e-90 24
reject -375.891395077 -375.891395077 5744.60417871 0.0 70
reject -375.891395077 -375.891395077 5338.30046239 0.0 95
reject -375.891395077 -375.891395077 7863.81532011 0.0 58
reject -375.891395077 -375.891395077 8909.40820889 0.0 97
reject -375.891395077 -375.891395077 8550.29109758 0.0 72
reject -375.891395077 -375.891395077 899.09339137 0.0 14
reject -375.891395077 -375.891395077 30.6630177671 4.8219264894e-12 37
reject -375.891395077 -375.891395077 667.87271919 8.84226328037e-289 9
reject -375.891395077 -375.891395077 58.0118187381 6.39421502805e-24 41
reject -375.891395077 -375.891395077 1206.74012 0.0 53
reject -375.891395077 -375.891395077 3224.99581142 0.0 9
reject -375.891395077 -375.891395077 745.200877817 0.0 93
reject -375.891395077 -375.891395077 2.52185262929 8.03106829181 31
reject -375.891395077 -375.891395077 17.8583585173 1.7547

In [11]:
aa_num=10
aa_name='A'

In [12]:
test=pose_from_sequence(aa_num*aa_name)

In [13]:
print test

PDB file name: AAAAAAAA
Total residues:10
Sequence: AAAAAAAAAA
Fold tree:
FOLD_TREE  EDGE 1 10 -1 


In [10]:
current_angle=pose.phi(39)
print current_angle

RuntimeError: unidentifiable C++ exception

In [ ]:
'''
HW4:
1. Fold a 10-mer poly-alanine using 100 independent trajectories, using any variant of the
folding algorithm that you like. (A trajectory is a path through the conformation space
traveled during the calculation. The end result of each independent trajectory is called a
“decoy”. Given enough sampling, the lowest energy decoy may correspond to the
global minimum.) Create a Ramachandran plot using the lowest-scoring conformations
(decoys) from all 100 independent trajectories. Repeat this for a 10-mer poly-glycine.
How do the plots differ? Compare with the plots in Richardson’s article.

2. Test your folding program’s ability to predict a real fold from scratch. Choose a small
protein to keep the computation time down, such as Hox-B1 homeobox protein (1B72)
or RecA (2REB). How many iterations and how many independent trajectories do you
need to run to find a good structure?'''

In [53]:
pose_name='10*A'

In [56]:
if '*' in pose_name:
    print 'yes'

yes


In [14]:
l=["A",1,'10']
print l

['A', 1, '10']


In [17]:
g=open('test.txt','w')
for i in l:
    g.write(str(i)+'\n')
g.close()

In [18]:
pwd


u'/Users/XT/Dropbox/Research/ComBioCourse/WorkshopNote'

In [19]:
ls

10ALA_100decoys_angles.txt      1B72.pdb                        Workshop#3_Scoring.ipynb
10ALA_100decoys_angles_500.txt  1B72_100decoys_angles.txt       Workshop4.ipynb
10GLY_100decoys_angles_500.txt  1B72_100decoys_angles_500.txt
1B72.clean.pdb                  Python Primer.ipynb


In [29]:
pose_name='1B72'
g=open(pose_name+'.clean.pdb','r')
for line in g.readlines():
    if len(line.split()[3]):
        print line.split()

['ATOM', '1', "O5'", 'DA', 'D', '1', '9.384', '4.091', '14.803', '1.00', '97.43', 'O']
['ATOM', '2', "C5'", 'DA', 'D', '1', '9.435', '3.410', '16.059', '1.00', '96.47', 'C']
['ATOM', '3', "C4'", 'DA', 'D', '1', '10.544', '3.912', '16.955', '1.00', '96.02', 'C']
['ATOM', '4', "O4'", 'DA', 'D', '1', '11.828', '3.679', '16.318', '1.00', '94.39', 'O']
['ATOM', '5', "C3'", 'DA', 'D', '1', '10.487', '5.411', '17.264', '1.00', '96.17', 'C']
['ATOM', '6', "O3'", 'DA', 'D', '1', '10.851', '5.661', '18.626', '1.00', '97.79', 'O']
['ATOM', '7', "C2'", 'DA', 'D', '1', '11.529', '6.001', '16.333', '1.00', '94.57', 'C']
['ATOM', '8', "C1'", 'DA', 'D', '1', '12.561', '4.892', '16.266', '1.00', '92.47', 'C']
['ATOM', '9', 'N9', 'DA', 'D', '1', '13.337', '4.911', '15.029', '1.00', '88.80', 'N']
['ATOM', '10', 'C8', 'DA', 'D', '1', '12.888', '5.140', '13.750', '1.00', '87.81', 'C']
['ATOM', '11', 'N7', 'DA', 'D', '1', '13.841', '5.161', '12.851', '1.00', '86.40', 'N']
['ATOM', '12', 'C5', 'DA', 'D', '1'

In [44]:
pose.total_residue()

181

In [45]:
print pose.residue(4).name()

CYT


In [43]:
pose=pose_from_file('1B72.clean.pdb')

core.conformation.Conformation: [ WARNING ] Building missing atom ( P  ) at root of residue tree, using stubs:  O5'  C5'  C4'
This probably means that a torsion angle is being taken from the ideal residue and
should be further optimized...
core.conformation.Conformation: [ WARNING ] Building missing atom ( P  ) at root of residue tree, using stubs:  O5'  C5'  C4'
This probably means that a torsion angle is being taken from the ideal residue and
should be further optimized...
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ARG 42
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue ARG 42
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE  on residue ARG 42
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CZ  on residue ARG 42
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NH1 on residue ARG 42
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NH2 on residue ARG 42
core

In [42]:
pose=Pose()